In [1]:
import torch
from torch import Tensor

In [8]:
bs = 4
dim = 3
num_classes = 5

In [9]:
features = torch.randint(0, 5, (bs, dim))
print("features: ", features)
classifier = torch.randint(5, 10, (dim, num_classes))
print("classifier: ", classifier)

features:  tensor([[0, 3, 1],
        [1, 1, 4],
        [4, 1, 0],
        [1, 0, 4]])
classifier:  tensor([[9, 7, 8, 5, 7],
        [9, 9, 7, 9, 6],
        [5, 8, 5, 8, 8]])


In [19]:
classifier.T

tensor([[9, 5, 8],
        [9, 5, 5],
        [7, 7, 5],
        [8, 5, 6]])

In [10]:
logits = features @ classifier
logits

tensor([[32, 35, 26, 35, 26],
        [38, 48, 35, 46, 45],
        [45, 37, 39, 29, 34],
        [29, 39, 28, 37, 39]])

In [11]:
val, ind = torch.topk(logits, 2, dim=-1)
print(val)
print(ind)

tensor([[35, 35],
        [48, 46],
        [45, 39],
        [39, 39]])
tensor([[1, 3],
        [1, 3],
        [0, 2],
        [1, 4]])


In [12]:
print(classifier)
print(ind)

tensor([[9, 7, 8, 5, 7],
        [9, 9, 7, 9, 6],
        [5, 8, 5, 8, 8]])
tensor([[1, 3],
        [1, 3],
        [0, 2],
        [1, 4]])


In [25]:
ind[0, :]

tensor([1, 2])

In [ ]:
topk_class_embeds = torch.stack([torch.index_select(classifier, dim=1, index=ind[i, :]).T for i in range(bs)])
print(topk_class_embeds.shape) # bs x topk x dim
print(topk_class_embeds)

torch.Size([4, 2, 3])
tensor([[[7, 9, 8],
         [5, 9, 8]],

        [[7, 9, 8],
         [5, 9, 8]],

        [[9, 9, 5],
         [8, 7, 5]],

        [[7, 9, 8],
         [7, 6, 8]]])


In [14]:
ff = features.unsqueeze(1).expand(-1, 2, -1)
print(features)
print(ff.shape)
print(ff)

tensor([[0, 3, 1],
        [1, 1, 4],
        [4, 1, 0],
        [1, 0, 4]])
torch.Size([4, 2, 3])
tensor([[[0, 3, 1],
         [0, 3, 1]],

        [[1, 1, 4],
         [1, 1, 4]],

        [[4, 1, 0],
         [4, 1, 0]],

        [[1, 0, 4],
         [1, 0, 4]]])


In [15]:
y = torch.cat([ff, topk_class_embeds], dim=-1)
print(y.shape)
print(y)

torch.Size([4, 2, 6])
tensor([[[0, 3, 1, 7, 9, 8],
         [0, 3, 1, 5, 9, 8]],

        [[1, 1, 4, 7, 9, 8],
         [1, 1, 4, 5, 9, 8]],

        [[4, 1, 0, 9, 9, 5],
         [4, 1, 0, 8, 7, 5]],

        [[1, 0, 4, 7, 9, 8],
         [1, 0, 4, 7, 6, 8]]])


In [56]:
yy = y.view(bs * 2, -1)
yy = yy.to(torch.float32)
yy.dtype

torch.float32

In [60]:
y = y.to(torch.float32)
y.dtype
y.shape

torch.Size([4, 2, 6])

In [17]:
from torch import nn
itm_head = nn.Sequential(
                nn.Linear(6, 3),
                nn.ReLU(),
                nn.Linear(3, 2),
            )
print(itm_head)

Sequential(
  (0): Linear(in_features=6, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=2, bias=True)
)


In [ ]:
itm_head

AttributeError: 'Sequential' object has no attribute 'dtype'

In [19]:
out = itm_head(y.to(torch.float32))
print(out.shape)
print(out)

torch.Size([4, 2, 2])
tensor([[[-1.8628, -1.6898],
         [-1.8653, -1.6738]],

        [[-2.2409, -2.1965],
         [-2.2435, -2.1804]],

        [[-1.3921, -1.4457],
         [-1.4949, -1.4215]],

        [[-2.1500, -2.3126],
         [-2.1877, -1.9847]]], grad_fn=<ViewBackward0>)


In [21]:
print(out.shape)
print(out[:,:,0])

torch.Size([4, 2, 2])
tensor([[-1.8628, -1.8653],
        [-2.2409, -2.2435],
        [-1.3921, -1.4949],
        [-2.1500, -2.1877]], grad_fn=<SelectBackward0>)


In [22]:
max = torch.argmax(out[:, :, 0], dim=-1)
print(max.shape)
print(max)

torch.Size([4])
tensor([0, 0, 0, 0])


In [24]:
print(ind.shape)
print(ind)

torch.Size([4, 2])
tensor([[1, 3],
        [1, 3],
        [0, 2],
        [1, 4]])


In [29]:
max.unsqueeze(-1)

tensor([[0],
        [0],
        [0],
        [0]])

In [28]:
#class_indices = torch.index_select(ind, dim=1, index=max)
class_indices = torch.gather(ind, dim=1, index=max.unsqueeze(-1))
print(class_indices.shape)
print(class_indices)

torch.Size([4, 1])
tensor([[1],
        [1],
        [0],
        [1]])


In [ ]:
torch.argwhere()

In [18]:
topk_class_features = classifier[:, ind].permute(2, 0, 1)
topk_class_features

tensor([[[9, 7, 9, 9],
         [5, 7, 5, 5],
         [8, 5, 8, 8]],

        [[7, 9, 7, 8],
         [7, 5, 7, 5],
         [5, 5, 5, 6]]])

In [16]:
classifier_expanded = classifier.unsqueeze(0).expand(ind.shape[0], -1, -1)

# Gather values based on ind
selected = torch.gather(classifier_expanded, dim=2, index=ind.unsqueeze(1))

print(selected)

tensor([[[9, 7]],

        [[7, 9]],

        [[9, 7]],

        [[9, 8]]])


In [21]:
t = torch.tensor([[1, 2], [3, 4]])
t

tensor([[1, 2],
        [3, 4]])

In [23]:
b = torch.tensor([[0, 0], [1, 0]])
b

tensor([[0, 0],
        [1, 0]])

In [24]:
torch.gather(t, 1, b)

tensor([[1, 1],
        [4, 3]])

In [1]:
import torch

In [4]:
feature = torch.randint(0, 5, (2,4))
print(feature)
classifier = torch.randint(5, 10, (4, 10))
print(classifier)

tensor([[1, 4, 3, 2],
        [0, 2, 1, 3]])
tensor([[5, 7, 9, 9, 9, 9, 9, 9, 5, 5],
        [8, 8, 8, 5, 6, 9, 7, 7, 7, 7],
        [5, 5, 9, 9, 6, 6, 7, 7, 6, 8],
        [6, 5, 5, 8, 7, 9, 6, 9, 5, 5]])


In [5]:
logits = feature @ classifier
print(logits)

tensor([[64, 64, 78, 72, 65, 81, 70, 76, 61, 67],
        [39, 36, 40, 43, 39, 51, 39, 48, 35, 37]])


In [7]:
val, ind = torch.topk(logits, 4, dim=1)
print(val)
print(ind)

tensor([[81, 78, 76, 72],
        [51, 48, 43, 40]])
tensor([[5, 2, 7, 3],
        [5, 7, 3, 2]])


In [6]:
s = torch.index_select(classifier, dim=1, index=ind)
print(s)

tensor([[7, 8, 9, 8],
        [8, 8, 7, 5],
        [7, 5, 5, 9],
        [6, 7, 7, 8]])


In [7]:
feature.expand(4, -1)
print(feature)

tensor([0, 3, 1, 0])


In [8]:
x = torch.concatenate([feature.expand(4, -1), s.T], dim=-1)
print(x)

tensor([[0, 3, 1, 0, 7, 8, 7, 6],
        [0, 3, 1, 0, 8, 8, 5, 7],
        [0, 3, 1, 0, 9, 7, 5, 7],
        [0, 3, 1, 0, 8, 5, 9, 8]])


In [10]:
from torch import nn
itm_head = nn.Sequential(
                nn.Linear(8, 4),
                nn.ReLU(),
                nn.Linear(4, 2),
            )

In [11]:
y = itm_head(x.to(torch.float32))
y = y + torch.randn_like(y)
print(y)

tensor([[ 0.0516,  2.0922],
        [-1.2897,  0.3224],
        [-2.6128,  1.4645],
        [-1.8627,  3.6383]], grad_fn=<AddBackward0>)


In [12]:
max = torch.argmax(y[:, 0])
print(max)

tensor(0)


In [13]:
max_ind = ind[max]
print(max_ind)

tensor(3)


In [ ]:
def itm_eval(itm_head, logits, topk, features, classifier):
    val, ind = torch.topk(logits, topk, dim=1)
    topk_class_embeds = torch.stack([torch.index_select(classifier, dim=1, index=ind[i, :]).T for i in range(bs)])
    